In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples
import nltk
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
train_pos=all_positive_tweets[:4000]
train_neg=all_negative_tweets[:4000]
test_pos=all_positive_tweets[4000:]
test_neg=all_negative_tweets[4000:]

In [2]:
train_x=train_pos+train_neg
test_x=test_pos+test_neg
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y=np.append(np.ones((len(test_pos),1)),np.zeros((len(test_neg),1)),axis=0)

In [3]:
import re 
import string 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [103]:
def clean_tweets(tweets):
    ct=[]
    for t in tweets:
        t=re.sub(r'@\w*','',t)
        t=re.sub(r'#','',t)
        t=re.sub(r'http\S+','',t)
        t=re.sub(r'[$&\n]\w*','',t)
        t=re.sub(r'[.*]','',t)
#         t=re.sub(r'u','',t)
#         t=re.sub(r'ur','',t)
#         t=re.sub(r'[`^()-]','',t)
        ct.append(t)
    return ct
        

In [104]:
def tok_stem_tweets(clean_tweets):
    st=[]
    tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    stemmer=PorterStemmer()
    sw=stopwords.words('english')
    for t in clean_tweets:
        tokens=tokenizer.tokenize(t)
        stem_tweet=[]
        for word in tokens:
            if(word not in sw and word not in string.punctuation):
                stem_word=stemmer.stem(word)
                stem_tweet.append(stem_word)
        st.append(stem_tweet)
    return st
            

In [105]:
def gen_freq_table(stem_tweets,target):
    freq={}
    target=np.squeeze(target).tolist()
    for y,tweet in zip(target,stem_tweets):
        for word in tweet:
            pair=(word,y)
            if pair in freq:
                freq[pair]+=1
            else:
                freq[pair]=1
    return freq
    

In [106]:
def gen_features(stem_tweets,freq):
    feature_matrix=[]
    for t in stem_tweets:
        t_features=[1]
        pos=0
        neg=0
        for word in t:
            try:
                pos=pos+freq[(word,1)]
            except:
                None
            try:
                neg=neg+freq[(word,0)]
            except:
                None
        t_features.append(pos)
        t_features.append(neg)
        feature_matrix.append(t_features)
    return feature_matrix
                
            

In [119]:
cleanx1=clean_tweets(train_x)
stemx1=tok_stem_tweets(cleanx1)
freq=gen_freq_table(stemx1,train_y)
# freq
x1=gen_features(stemx1,freq)
cleanx2=clean_tweets(test_x)
stemx2=tok_stem_tweets(cleanx2)
x2=gen_features(stemx2,freq)


In [120]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

In [121]:
clf.fit(x1,train_y)

E:\programs\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\programs\anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [122]:
clf.score(x1,train_y)

0.994375

In [123]:
clf.score(x2,test_y)

0.995